In [13]:
import xml.etree.ElementTree as ET
import numpy as np
import os

from shared_utils.general import parse_float_seq, parse_int_seq, list_to_string, gen_models_folder_path

In [14]:
def flip_pos_array(pos_array, flip_func):
    for i in range(0, len(pos_array), 3):
        pos_array[i:i+3] = flip_func(pos_array[i:i+3])
    return pos_array

def flip_string(pos_string, flip_func, string_parser):
    return list_to_string(flip_pos_array(string_parser(pos_string), flip_func))

def flip_pos_string(pos_string, flip_func):
    return flip_string(pos_string, flip_func, parse_float_seq)

def flip_int_string(pos_string, flip_func):
    return flip_string(pos_string, flip_func, parse_int_seq)

def is_3_coord_pos(pos_string):
    return len(parse_float_seq(pos_string)) % 3 == 0

In [ ]:
MODEL_PATH = os.path.join(gen_models_folder_path(), "exo_with_patient", "nesm_with_patient.xml")
assert os.path.isfile(MODEL_PATH)

## Flip hand frames in v11

In [7]:
def flip_hand_pos(pos):
    pos_list = []
    pos_list.append(-pos[1])
    pos_list.append(pos[0])
    pos_list.append(pos[2])
    return pos_list

def flip_hand_string(pos_string):
    return flip_pos_string(pos_string, flip_hand_pos)

In [8]:
tree = ET.parse(MODEL_PATH)
root = tree.getroot()

checked_attrib = ['pos', 'fromto', 'axis']

hand = root.findall(f".//body[@name='hand_r']")[0]

for child in hand.iter():
    for attrib in checked_attrib:
        if attrib in child.attrib:
            child.attrib[attrib] = flip_hand_string(child.attrib[attrib])


# print(ET.tostring(hand, encoding='unicode'))

## Flip human body frames

In [15]:
def flip_body_pos(pos):
    flipped_pos = []
    flipped_pos.append(-pos[1])
    flipped_pos.append(pos[0])
    flipped_pos.append(pos[2])
    return flipped_pos

def flip_body_unsigned_pos(pos):
    return np.abs(flip_body_pos(pos))


def flip_body_pos_string(pos_string):
    return flip_pos_string(pos_string, flip_body_pos)

def flip_body_unsigned_pos_string(pos_string):
    return flip_pos_string(pos_string, flip_body_unsigned_pos)


def flip_body_int_string(int_string):
    return flip_int_string(int_string, flip_body_pos)

def flip_body_unsigned_int_string(int_string):
    return flip_int_string(int_string, flip_body_unsigned_pos)

In [20]:
tree = ET.parse(MODEL_PATH)
root = tree.getroot()

pos_attrib = ['pos', 'fromto', 'axis']

thorax = root.findall(f".//body[@name='thorax']")[0]

for child in thorax.iter():
    for attrib in pos_attrib:
        if attrib in child.attrib:
            child.attrib[attrib] = flip_body_pos_string(child.attrib[attrib])

    if 'size' in child.attrib and is_3_coord_pos(child.attrib['size']):
        child.attrib['size'] = flip_body_unsigned_pos_string(child.attrib['size'])

    if 'diaginertia' in child.attrib:
        child.attrib['diaginertia'] = flip_body_unsigned_pos_string(child.attrib['diaginertia'])

    if 'count' in child.attrib:
        child.attrib['count'] = flip_body_unsigned_int_string(child.attrib['count'])

# print(ET.tostring(thorax, encoding='unicode'))
